In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, IntSlider, FloatSlider

# 1. GENEROWANIE DANYCH SYNTETYCZNYCH
# Symulacja odczytów NDVI dla pola o powierzchni 50 ha (10 000 punktów pomiarowych)
np.random.seed(42)
ndvi_data = np.clip(np.random.normal(0.78, 0.05, 10000), 0.1, 0.95)
df = pd.DataFrame({'NDVI': ndvi_data})

# Model agronomiczny: Funkcja plonowania uwzględniająca krzywą reakcji na azot
def oblicz_plon(ndvi, dawka_azotu):
    # Bazowy potencjał plonotwórczy zależny od kondycji roślin (NDVI)
    potencjal = ndvi * 12

    # Wyznaczenie współczynnika wpływu nawożenia
    if dawka_azotu <= 190:
        # Poniżej optimum: liniowy wzrost plonu
        wsp = min(dawka_azotu / 160, 1.0)
    else:
        # Powyżej optimum: Kara za przenawożenie (wyleganie łanu)
        # Każde 10kg nadmiaru redukuje plon o 1%
        nadmiar = dawka_azotu - 190
        wsp = 1.0 - (nadmiar * 0.001)

    # Zwrócenie wyniku z uwzględnieniem losowej zmienności biologicznej (+/- 0.2 t/ha)
    return max(potencjal * wsp + np.random.normal(0, 0.2), 0)

# 2. DEFINICJA SCENARIUSZY AGROTECHNICZNYCH

# Scenariusz A: Tradycyjny (Jednolita dawka)
# Założenie: Stała, wysoka dawka 220 kg N/ha (ryzyko przenawożenia)
df['N_Tradycyjny'] = 220
df['Plon_Tradycyjny'] = df.apply(lambda x: oblicz_plon(x['NDVI'], 220), axis=1)

# Scenariusz B: Rolnictwo Precyzyjne (VRA)
# Algorytm zmiennego dawkowania w oparciu o strefy NDVI
def dobierz_dawke(ndvi):
    if ndvi < 0.70: return 140  # Strefa o niskim potencjale (oszczędność)
    elif ndvi > 0.82: return 170 # Strefa o wysokim potencjale (maksymalizacja)
    else: return 160            # Strefa standardowa

df['N_Precyzyjny'] = df['NDVI'].apply(dobierz_dawke)
df['Plon_Precyzyjny'] = df.apply(lambda x: oblicz_plon(x['NDVI'], x['N_Precyzyjny']), axis=1)


# 3. INTERAKTYWNY DASHBOARD EKONOMICZNY
def symulator_biznesowy(cena_pszenicy, cena_nawozu):

    # Kalkulacja marży brutto dla obu technologii
    zysk_A = (df['Plon_Tradycyjny'] * cena_pszenicy) - (df['N_Tradycyjny'] * cena_nawozu)
    zysk_B = (df['Plon_Precyzyjny'] * cena_pszenicy) - (df['N_Precyzyjny'] * cena_nawozu)

    srednia_A = zysk_A.mean()
    srednia_B = zysk_B.mean()
    roznica = srednia_B - srednia_A

    # Wizualizacja wyników
    plt.figure(figsize=(10, 6))

    bars = plt.bar(['Tradycyjna', 'Precyzyjna (VRA)'], [srednia_A, srednia_B],
                   color=['#e74c3c', '#2ecc71'], alpha=0.9)

    plt.axhline(0, color='black', linewidth=1)

    # Etykiety wartości
    plt.text(0, srednia_A + 100, f'{srednia_A:.0f} zł', ha='center', fontsize=14, weight='bold')
    plt.text(1, srednia_B + 100, f'{srednia_B:.0f} zł', ha='center', fontsize=14, weight='bold')

    plt.title(f'SYMULACJA ZYSKU (Różnica: +{roznica:.0f} zł/ha)', fontsize=16)
    plt.ylabel('Zysk operacyjny (zł/ha)')
    plt.ylim(0, max(srednia_B, srednia_A) * 1.2)
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.show()

# Uruchomienie interfejsu użytkownika
print("🎛️ PANEL ANALITYCZNY:")
interact(symulator_biznesowy,
         cena_pszenicy = IntSlider(min=600, max=1400, step=50, value=900, description='Cena Zboża'),
         cena_nawozu = FloatSlider(min=2.0, max=10.0, step=0.5, value=5.0, description='Cena Azotu'));